# Modeling Reversal Task

### RW model of the reversal task in the aging experiment

The aim of this notbook is to see if age affects appetative reversal learning.

participants have 70 trials 40% reinforced.

reversal of stimuli occurs after 35 trials.

This notbook is based on Or's simulation of SCR.

## load libraries

In [39]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import os

import pymc3 as pm
import arviz as az
import statsmodels.api as sm
import statsmodels.formula.api as smf

from glob import glob

## Get data

make sure only participant with complete data set are loaded

In [40]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv'

db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            
            db = db.append(df[df.trialNum<36])
    except:
        print(sub)
        print('error')

print('number of subject: ', len(db['sub'].unique()))

/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_60_RV/ETLearning_1638474572_60.csv
error
number of subject:  48


## get descriptive data

In [43]:
n_subj   = len(db['sub'].unique())
n_trials = max(db.trialNum)

trials, subj = np.meshgrid(range(n_trials), range(n_subj))
trials = tt.as_tensor_variable(trials.T)
subj   = tt.as_tensor_variable(subj.T)

In [50]:
stim   = np.reshape([db['rectOri']],   (n_subj, n_trials)).T
reward = np.reshape([db['rectValue']], (n_subj, n_trials)).T
rating = np.reshape([db['rating']],    (n_subj, n_trials)).T

stim   = np.array(stim/45,  dtype='int')
reward = np.array(reward/6*9, dtype='int')

In [51]:
stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

# create a pymc3 model

In [52]:
 
# generate functions to run
def update_Q(stim, reward,
             Qs,vec,
             alpha, n_subj):
    """
    This function updates the Q table according to the RL update rule.
    It will be called by theano.scan to do so recursevely, given the observed data and the alpha parameter
    This could have been replaced be the following lamba expression in the theano.scan fn argument:
        fn=lamba action, reward, Qs, alpha: tt.set_subtensor(Qs[action], Qs[action] + alpha * (reward - Qs[action]))
    """
     
    PE = reward - Qs[tt.arange(n_subj), stim]
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + alpha * PE)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    return Qs, vec

In [53]:
# try alpha as beta distribution
with pm.Model() as mB:
    
   # betaHyper= pm.Normal('betaH', 0, 1)
    alpha = pm.Beta('alpha', 1,1, shape=n_subj)
    beta = pm.Normal('beta',0, 5, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trB = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, beta, alpha]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 190 seconds.


In [54]:
az.summary(trB, var_names='alpha')[:20]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.053,0.029,0.001,0.102,0.001,0.000,2228.0,1069.0,1.0
alpha[1],0.039,0.018,0.004,0.071,0.000,0.000,2708.0,1823.0,1.0
alpha[2],0.027,0.016,0.000,0.055,0.000,0.000,2539.0,1715.0,1.0
alpha[3],0.079,0.029,0.026,0.133,0.000,0.000,3618.0,2237.0,1.0
alpha[4],0.022,0.016,0.000,0.051,0.000,0.000,2517.0,1382.0,1.0
alpha[5],0.038,0.028,0.000,0.088,0.000,0.000,2878.0,1887.0,1.0
alpha[6],0.013,0.011,0.000,0.034,0.000,0.000,3264.0,1876.0,1.0
alpha[7],0.118,0.040,0.042,0.192,0.001,0.000,3463.0,2561.0,1.0
alpha[8],0.052,0.034,0.000,0.111,0.001,0.000,2600.0,1454.0,1.0
alpha[9],0.027,0.020,0.000,0.062,0.000,0.000,3233.0,1987.0,1.0


In [55]:
# try with intercept
with pm.Model() as mB_I:
    
   # betaHyper= pm.Normal('betaH', 0, 1)
    intercept = pm.Normal('intercept', 0, 5)
    
    alpha = pm.Beta('alpha', 1,1, shape=n_subj)
    beta = pm.Normal('beta',0, 5, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trB_I = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, beta, alpha, intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 185 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [56]:
az.summary(trB_I, var_names='alpha')[:20]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.587,0.262,0.127,0.999,0.004,0.003,5281.0,2475.0,1.0
alpha[1],0.108,0.083,0.000,0.225,0.002,0.002,4774.0,2296.0,1.0
alpha[2],0.143,0.098,0.001,0.317,0.001,0.001,3693.0,2026.0,1.0
alpha[3],0.244,0.098,0.077,0.432,0.001,0.001,4145.0,1746.0,1.0
alpha[4],0.108,0.083,0.000,0.259,0.001,0.001,3915.0,2231.0,1.0
alpha[5],0.485,0.226,0.068,0.907,0.004,0.003,3476.0,2312.0,1.0
alpha[6],0.335,0.285,0.000,0.846,0.005,0.004,3453.0,3093.0,1.0
alpha[7],0.400,0.140,0.139,0.654,0.003,0.002,3174.0,1862.0,1.0
alpha[8],0.491,0.284,0.035,0.968,0.003,0.003,7974.0,1964.0,1.0
alpha[9],0.447,0.283,0.004,0.911,0.004,0.003,4207.0,2582.0,1.0


In [57]:
az.compare({'model1': trB, 'model2':trB_I})

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/stats/stats.py:695: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
model1,0,-4003.300486,79.241475,0.000000,0.87918,25.815531,0.000000,True,log
model2,1,-4044.117654,74.104098,40.817168,0.12082,23.204206,10.384456,False,log


In [58]:
# try with intercept
with pm.Model() as mB_Is:
    
   # betaHyper= pm.Normal('betaH', 0, 1)
    intercept = pm.Normal('intercept', 0, 5, shape=n_subj)
    
    alpha = pm.Beta('alpha', 1,1, shape=n_subj)
    beta = pm.Normal('beta',0, 5, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec],
        non_sequences=[alpha, n_subj])
   
    
    vec_ = vec[trials,subj,0] * beta[subj] + intercept[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    trB_Is = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, beta, alpha, intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 953 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [59]:
az.summary(trB_Is, var_names='alpha')[:20]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],0.346,0.323,0.000,0.904,0.007,0.005,2101.0,1927.0,1.00
alpha[1],0.139,0.208,0.000,0.623,0.004,0.003,2991.0,2270.0,1.00
alpha[2],0.274,0.241,0.000,0.757,0.004,0.003,2035.0,1643.0,1.00
alpha[3],0.164,0.124,0.010,0.374,0.002,0.002,3326.0,2118.0,1.00
alpha[4],0.289,0.263,0.000,0.811,0.004,0.003,2272.0,2127.0,1.00
alpha[5],0.399,0.239,0.000,0.819,0.005,0.003,2047.0,1249.0,1.01
alpha[6],0.334,0.278,0.000,0.861,0.006,0.004,1341.0,892.0,1.00
alpha[7],0.274,0.144,0.032,0.527,0.002,0.002,3283.0,1495.0,1.00
alpha[8],0.292,0.281,0.000,0.847,0.005,0.003,2323.0,2188.0,1.00
alpha[9],0.388,0.295,0.000,0.896,0.007,0.005,1193.0,1134.0,1.00


In [60]:
az.compare({'model1': trB, 'model2':trB_I, 'model3':trB_Is})

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/stats/stats.py:695: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
model3,0,-4000.762514,104.953049,0.000000,0.570773,25.550799,0.000000,False,log
model1,1,-4003.300486,79.241475,2.537972,0.429227,25.815531,6.872247,True,log
model2,2,-4044.117654,74.104098,43.355140,0.000000,23.204206,8.927458,False,log


## hierarchal model

In [ ]:
# try alpha as beta distribution
with pm.Model() as m_H:
    
    # intercept
    mu = pm.Normal('mu', 0, 1)
    sd = pm.HalfNormal('sd',5) 
    intercept_matt = pm.Normal('intercept_matt', mu=0, sd=1, shape=n_subj)
    intercept = pm.Deterministic('intercept',intercept_matt + mu*sd)
    
    phi = pm.Uniform("phi", lower=0.0, upper=1.0)

    kappa_log = pm.Exponential("kappa_log", lam=1.5)
    kappa = pm.Deterministic("kappa", tt.exp(kappa_log))

    alpha = pm.Beta("alpha", alpha=phi * kappa, beta=(1.0 - phi) * kappa, shape=n_subj)
    
    
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 1)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
       
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec0 = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec], updates = theano.scan(
        fn=update_Q,
        sequences=[stim, reward],
        outputs_info=[Qs, vec0],
        non_sequences=[alpha, n_subj])
   
     
    vec_ = vec[trials, subj,0] * beta[subj] + intercept[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    
    tr_hB = pm.sample(target_accept=.9, chains=4, cores=8, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 8 jobs)
NUTS: [eps, beta, beta_sd, beta_h, alpha, kappa_log, phi, intercept_matt, sd, mu]


In [ ]:
az.summary(tr_hB, var_names='alpha')[:10]

## model comparison

In [ ]:
comp = az.compare({'model1':trB, 'model2': tr_hB}, ic='loo')
comp

In [ ]:
az.plot_compare(comp)

## Correlate expected value and subject data

In [ ]:
a = trB.posterior.stack(draws=('chain','draw'))
a = a.expected_value
mean_a = np.mean(a, axis=2)
mean_a.shape

In [ ]:
for i in np.arange(10):
    cor1 = scipy.stats.pearsonr(rating[:,i], mean_a[:,i])
    print(cor1)

## The Pearce-Hall Hybrid model

This is Or's attempt to build the PH Hybrid model. This model doesn't assume a simple constant learning rate (as the RW), rather, it incorporated both a constant learning rate and a dynamic one. The dynamic one is being updated by the amount of new information given. The model goes like that: 

(1) Vi(k+1) = Vi (k) + (k) + κα(k)δ

(2) δ = shock - Vi(k) 

(3) α(k+1) = μ|δ| + (1-μ)α(k)

So the current value is an update of the previous one plus a constant learning rate (kappa) and an associability weight (alpha) (times the delta = prediction error).

The α is set by a constant weight of associability (eta) and the previous α.

So now, our updating function will include those elements as well

In [59]:
# generate functions to run
def update_Q_hb(stim, shock,
             Qs,vec,alpha,assoc,
             eta,kappa, n_subj):
    """
    This function updates the Q table according to Hybrid PH model
    For information, please see this paper: https://www.sciencedirect.com/science/article/pii/S0896627316305840?via%3Dihub
  
    """
      
    delta = shock - Qs[tt.arange(n_subj), stim]
    alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta * abs(delta) + (1-eta)*alpha[tt.arange(n_subj), stim])
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + kappa*alpha[tt.arange(n_subj), stim] * delta)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    # we use the same idea to get the associability per trial
    assoc = tt.set_subtensor(assoc[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                alpha[tt.arange(n_subj),1], alpha[tt.arange(n_subj),0])))
    
    return Qs, vec, alpha, assoc

In [60]:
with pm.Model() as m:
  
    # hyperpriors for eta and kappa
    phi = pm.Uniform("phi", lower=0.0, upper=1.0, shape=2)
    
    # κ   
    k_log1 = pm.Exponential("k_log1", lam=1.5)
    k1 = pm.Deterministic("k1", tt.exp(k_log1))
    kappa = pm.Beta("kappa", alpha=phi[0] * k1, beta=(1.0 - phi[0]) * k1, shape=n_subj)
    
    # β
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 5)
    beta = pm.Normal('beta',beta_h, beta_sd, shape=n_subj)
    
    # η
    k_log2 = pm.Exponential("k_log2", lam=1.5)
    k2 = pm.Deterministic("k2", tt.exp(k_log2))
    eta = pm.Beta('η', alpha=phi[1] * k2, beta=(1.0 - phi[1]) * k2, shape=n_subj)
    
   # kappa = pm.Beta('kappa', 1,1, shape=n_subj)
   # eta = pm.Beta('eta', 1,1, shape=n_subj)
    
  #  beta = pm.Normal('beta',0, 1, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 4.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 4.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64')
    assoc = 0 * tt.ones((n_subj,1), dtype='float64')
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_Q_hb,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
   
    
    vec_ = vec[trials, subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    # add associabillity
    #assoc = pm.Deterministic('alpha', assoc)
    
    tr = pm.sample(target_accept=.9, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [eps, η, k_log2, beta, beta_sd, beta_h, kappa, k_log1, phi]


/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/nachshon/anaconda3/envs/reversal/lib/python3.9/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 787 seconds.
There were 148 divergences after tuning. Increase `target_accept` or reparameterize.
There were 92 divergences 

In [61]:
az.summary(tr, var_names='η')[:10]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
η[0],0.812,0.253,0.258,1.0,0.009,0.007,333.0,497.0,1.01
η[1],0.797,0.272,0.174,1.0,0.012,0.008,289.0,643.0,1.02
η[2],0.833,0.246,0.288,1.0,0.009,0.007,275.0,307.0,1.02
η[3],0.859,0.201,0.436,1.0,0.007,0.006,224.0,374.0,1.02
η[4],0.834,0.242,0.284,1.0,0.011,0.007,298.0,595.0,1.01
η[5],0.859,0.220,0.377,1.0,0.011,0.008,284.0,494.0,1.02
η[6],0.826,0.252,0.260,1.0,0.012,0.009,245.0,417.0,1.02
η[7],0.856,0.209,0.396,1.0,0.008,0.006,331.0,483.0,1.02
η[8],0.829,0.236,0.322,1.0,0.011,0.008,272.0,780.0,1.02
η[9],0.812,0.260,0.218,1.0,0.013,0.009,231.0,531.0,1.02
